<a href="https://colab.research.google.com/github/ss8319/Colab-Notebooks/blob/main/VocabTransfer_openNMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Tokenize parent and child corpus
2.Build parent model
3.Build child model

In [ ]:
# Confirm GPU availability
# Runtime>Change runtime type
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
! pip install transformers
! pip install simpletransformers
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 6.6 MB 37.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 4.2 MB/s 
     |████████████████████████████████| 9.1 MB 47.1 MB/s 
     |████████████████████████████████| 1.8 MB 41.1 MB/s 
     |████████████████████████████████| 362 kB 43.6 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 44.7 MB/s 
     |████████████████████████████████| 145 kB 46.9 MB/s 
     |███████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%writefile step1.py
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
#step 1 - Create SentencePiece vocabulary for dataset
import sentencepiece as spm
import argparse


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", type=str, required=True)
    parser.add_argument("--prefix", type=str, required=True)
    parser.add_argument("--vocab_size", type=int, required=True)
    return parser.parse_args()


def create_vocab(dataset_pth, prefix, vocab_size):
    """output: {prefix}.model {prefix}.vocab"""
    print(dataset_pth, prefix, vocab_size)
    spm.SentencePieceTrainer.train(
        f"--input={dataset_pth} --model_prefix={prefix}  --vocab_size={vocab_size} \
        --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 \
        --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS] \
        --user_defined_symbols=[CLS],[SEP],[MASK]"
    )


def main():
    args = parse_args()
    create_vocab(args.dataset, args.prefix, args.vocab_size)


if __name__ == "__main__":
    main()

Writing step1.py


In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/scripture/oro-HARX_2022_01_29.txt --prefix=oro-HARX --vocab_size=8000

/content/gdrive/Shareddrives/Gutenberg/MT/scripture/oro-HARX_2022_01_29.txt oro-HARX 8000
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/content/gdrive/Shareddrives/Gutenberg/MT/scripture/oro-HARX_2022_01_29.txt --model_prefix=oro-HARX  --vocab_size=8000         --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3         --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS]         --user_defined_symbols=[CLS],[SEP],[MASK]
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/gdrive/Shareddrives/Gutenberg/MT/scripture/oro-HARX_2022_01_29.txt
  input_format: 
  model_prefix: oro-HARX
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1

In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/scripture/ko-BTKX_2021_10_14.txt --prefix=ko-BTKX --vocab_size=8000

/content/gdrive/Shareddrives/Gutenberg/MT/scripture/ko-BTKX_2021_10_14.txt ko-BTKX 8000
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/content/gdrive/Shareddrives/Gutenberg/MT/scripture/ko-BTKX_2021_10_14.txt --model_prefix=ko-BTKX  --vocab_size=8000         --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3         --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS]         --user_defined_symbols=[CLS],[SEP],[MASK]
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/gdrive/Shareddrives/Gutenberg/MT/scripture/ko-BTKX_2021_10_14.txt
  input_format: 
  model_prefix: ko-BTKX
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  spl

In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/corpora/en-CCMatrix_en_ko_clean_5k.txt --prefix=en-CCMatrix --vocab_size=5000

/content/gdrive/Shareddrives/Gutenberg/MT/corpora/en-CCMatrix_en_ko_clean_5k.txt en-CCMatrix 5000
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/content/gdrive/Shareddrives/Gutenberg/MT/corpora/en-CCMatrix_en_ko_clean_5k.txt --model_prefix=en-CCMatrix  --vocab_size=5000         --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3         --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS]         --user_defined_symbols=[CLS],[SEP],[MASK]
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/gdrive/Shareddrives/Gutenberg/MT/corpora/en-CCMatrix_en_ko_clean_5k.txt
  input_format: 
  model_prefix: en-CCMatrix
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  sp

In [ ]:
!python3 step1.py --dataset=/content/gdrive/Shareddrives/Gutenberg/MT/corpora/ko-CCMatrix_en_ko_clean_5k.txt --prefix=ko-CCMatrix --vocab_size=5000

/content/gdrive/Shareddrives/Gutenberg/MT/corpora/ko-CCMatrix_en_ko_clean_5k.txt ko-CCMatrix 5000
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/content/gdrive/Shareddrives/Gutenberg/MT/corpora/ko-CCMatrix_en_ko_clean_5k.txt --model_prefix=ko-CCMatrix  --vocab_size=5000         --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3         --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS]         --user_defined_symbols=[CLS],[SEP],[MASK]
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/gdrive/Shareddrives/Gutenberg/MT/corpora/ko-CCMatrix_en_ko_clean_5k.txt
  input_format: 
  model_prefix: ko-CCMatrix
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  sp

In [ ]:
!pip install python-dotenv
!pip install sacrebleu
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import sacrebleu

# Tell the SIL NLP tools where to find the Gutenberg resources
os.environ['SIL_NLP_DATA_PATH'] = "/content/gdrive/Shareddrives/Gutenberg"

In [ ]:
#uncomment to prepare df for trg->src also
import os
import pandas
def prepare_translation_datasets(data_path, src_lang, src, trg_lang, trg, test_size):
    src_text = []
    trg_text = []
    with open(os.path.join(data_path, src), "r", encoding="utf-8") as s, \
         open(os.path.join(data_path, trg), "r", encoding="utf-8") as t:
        for src_line, trg_line in zip(s, t):
            src_line = src_line.strip()
            trg_line = trg_line.strip()
            if len(src_line) == 0 or len(trg_line) == 0:
                continue
            src_text.append(src_line)
            trg_text.append(trg_line)

    X_train, X_test, y_train, y_test = train_test_split(src_text, trg_text, test_size=test_size)


    data = []
    for src, trg in zip(X_train, y_train):
        data.append([f"translate {src_lang} to {trg_lang}", src, trg])
        #data.append([f"translate {trg_lang} to {src_lang}", trg, src])

    train_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])


    data = []
    for src, trg in zip(X_test, y_test):
        data.append([f"translate {src_lang} to {trg_lang}", src, trg])
        #data.append([f"translate {trg_lang} to {src_lang}", trg, src])

    eval_df = pd.DataFrame(data, columns=["prefix", "input_text", "target_text"])

    return train_df, eval_df
    